In [1]:
%%bash

pip3 install pandas numpy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 KB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 KB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 KB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 11.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def compare_resource_usage(query, run, fname):
    memory_q1_opt = pd.read_csv(f'{query}/opt/resource-usage-{run}.csv')
    memory_q1_ref = pd.read_csv(f'{query}/ref/resource-usage-{run}.csv')
    
    plt.plot(memory_q1_ref['memory_used'] / (1000 * 1000 * 1000))
    plt.plot(memory_q1_opt['memory_used'] / (1000 * 1000 * 1000))
    plt.savefig(f'memory-{fname}.eps')
    plt.show()

    plt.plot(memory_q1_ref['cpu_used'])
    plt.plot(memory_q1_opt['cpu_used'])
    plt.show()

    N = 15
    plt.plot(np.convolve(memory_q1_ref['cpu_used'], np.ones(N)/N, mode='valid'))
    plt.plot(np.convolve(memory_q1_opt['cpu_used'], np.ones(N)/N, mode='valid'))
    plt.savefig(f'cpu15s-{fname}.eps')
    plt.show()

In [ ]:
compare_resource_usage('benchmark-results-lsqb/q1.sql', '05', 'q1-sf100')

#compare_resource_usage('benchmark-results-lsqb-sf100/q4.sql')

In [1]:
def gen_stats(resdir, ignore_first = True, results='opt'):
    df_opt = pd.read_csv(f'{resdir}/results-{results}.csv')
    if ignore_first:
        df_opt = df_opt[df_opt['run'] != 1]
    df_opt = df_opt.groupby(['query', 'physical_cj']).agg({'runtime': ['mean', 'std', 'sum']})
    df_opt.to_csv(f'{resdir}/results-{results}-mean.csv')
    return df_opt
def gen_stats_ref(resdir):
    stats_opt_df = pd.read_csv(f'{resdir}/results-ref.csv')
    df_opt = stats_opt_df[stats_opt_df['run'] != 1]
    df_opt = df_opt.groupby(['query']).agg({'runtime': ['mean', 'std', 'sum']})
    df_opt.to_csv(f'{resdir}/results-ref-mean.csv')
    return df_opt
def gen_stats_imdb(resdir):
    stats_opt_df = pd.read_csv(f'{resdir}/results-opt.csv')
    df_opt = stats_opt_df[stats_opt_df['run'] != 1]
    df_opt['query'] = df_opt['query'].replace(r'[a-z].sql', '.sql', regex=True)
    df_opt = df_opt.groupby(['query']).agg({'runtime': ['mean', 'std', 'sum']})
    df_opt.to_csv(f'{resdir}/results-opt-mean.csv')
    return df_opt

In [26]:
df_opt = gen_stats('benchmark-results-stats')
df_opt.sum()

runtime  mean     77.824798
         std      19.531498
         sum     389.123991
dtype: float64

In [27]:
df_opt = gen_stats('benchmark-results-stats-hint')
df_opt.sum()

runtime  mean     57.936630
         std       5.402731
         sum     289.683150
dtype: float64

In [27]:
pd.set_option('display.max_rows', None)

In [56]:
df_opt = gen_stats('benchmark-results-imdb', ignore_first = False)
#df_opt['query'] = df_opt['query'].replace(r'\d\\.sql', '.sql', regex=True)
df_opt

runtime                       
                                      mean        std         sum
query               physical_cj                                  
10a-unguarded-2.sql True          8.620857   2.003390   17.241713
10a-unguarded-3.sql True          9.919663   0.043037   19.839326
10a-unguarded-4.sql True          7.445304   2.488442   14.890609
10a-unguarded-5.sql True          8.017644   2.776277   16.035289
11a-unguarded-2.sql True          5.785589   0.065313   11.571179
11a-unguarded-3.sql True          5.840309   0.642704   11.680618
11a-unguarded-4.sql True          8.668871   0.203665   17.337741
11a-unguarded-5.sql True          6.094734   0.180450   12.189468
11a-unguarded-6.sql True          7.077333   0.527993   21.231999
12a-unguarded-2.sql True         22.604516   0.141099   45.209031
12a-unguarded-3.sql True         26.513610   1.129923   53.027220
12a-unguarded-4.sql True         25.842623   0.269888   51.685245
12a-unguarded-5.sql True         27.251851   0.785090   54.503702
12a-unguarded-6.sql True         28.992384   0.795900   57.984768
13a-unguarded-2.sql True         38.145303   0.755204   76.290606
13a-unguarded-3.sql True         41.462779   1.728428   82.925558
13a-unguarded-4.sql True         29.436788   0.579433   58.873575
13a-unguarded-5.sql True         34.874210   1.462016   69.748421
13a-unguarded-6.sql True         34.635201   1.803636   69.270402
14a-unguarded-2.sql True          8.577593   0.288746   17.155187
14a-unguarded-3.sql True          8.859225   0.089177   17.718450
14a-unguarded-4.sql True          8.269344   0.157789   16.538688
14a-unguarded-5.sql True         11.392181   0.142381   22.784362
14a-unguarded-6.sql True          9.445140   0.334737   18.890279
15a-unguarded-2.sql True          8.534110   0.536918   17.068220
15a-unguarded-3.sql True          7.584229   2.777802   15.168458
15a-unguarded-4.sql True          7.080052   2.143329   14.160104
15a-unguarded-5.sql True         10.718936   1.182072   21.437871
15a-unguarded-6.sql True          6.945267   2.073282   13.890534
16a-unguarded-2.sql True         49.605159  19.634838   99.210317
16a-unguarded-3.sql True         63.682727   3.654268  127.365453
16a-unguarded-4.sql True         70.292636   5.636997  140.585272
16a-unguarded-5.sql True         80.458133   5.783902  160.916266
16a-unguarded-6.sql True         78.322031   2.420909  156.644062
17a-unguarded-2.sql True         38.480951   0.241154   76.961901
17a-unguarded-3.sql True         52.357776  16.367695  104.715553
17a-unguarded-4.sql True         46.327928   9.115740   92.655855
17a-unguarded-5.sql True         49.104682   0.842367   98.209364
18a-unguarded-2.sql True         42.062753   2.842985   84.125505
18a-unguarded-3.sql True         35.143701   0.290398   70.287402
18a-unguarded-4.sql True         40.570921   2.292467   81.141842
18a-unguarded-5.sql True         47.170008  14.879875   94.340017
19a-unguarded-2.sql True         21.142730   1.597233   42.285460
19a-unguarded-3.sql True         22.052984   1.971788   44.105968
19a-unguarded-4.sql True         24.768657   1.365658   49.537314
19a-unguarded-5.sql True         24.037458   0.726359   48.074915
19a-unguarded-6.sql True         23.901627   2.963647   47.803254
1a-unguarded-2.sql  True         17.043378   2.430228   34.086757
1a-unguarded-3.sql  True         17.702384   3.096245   35.404768
1a-unguarded-4.sql  True         17.569714   1.516763   35.139428
20a-unguarded-2.sql True         25.900968   6.940010   51.801937
20a-unguarded-3.sql True         20.672974   0.523131   41.345947
20a-unguarded-4.sql True         25.104408   0.556273   50.208816
20a-unguarded-5.sql True         26.570231   3.587346   53.140462
20a-unguarded-6.sql True         26.135953   0.443185   52.271906
21a-unguarded-5.sql True          7.466307   0.387887   14.932615
21a-unguarded-6.sql True          8.005021   0.716941   16.010042
22a-unguarded-2.sql True          8.544030   0.516561   17.088060
22a-ungua

In [57]:
df_ref = gen_stats('benchmark-results-imdb', results='ref', ignore_first = False)
df_ref

runtime                       
                                       mean        std         sum
query               physical_cj                                   
10a-unguarded-2.sql False          6.173972   0.044003   12.347944
10a-unguarded-3.sql False          6.548627   0.128008   13.097255
10a-unguarded-4.sql False          5.638836   0.459086   11.277672
10a-unguarded-5.sql False          7.104491   0.239550   14.208982
11a-unguarded-2.sql False          5.831421   0.127936   11.662842
11a-unguarded-3.sql False          6.092713   0.345270   12.185427
11a-unguarded-4.sql False          6.308288   1.045292   12.616575
11a-unguarded-5.sql False          7.242555   0.840189   14.485111
11a-unguarded-6.sql False          7.129232   0.082742   14.258464
12a-unguarded-2.sql False         27.494178   1.776018   54.988356
12a-unguarded-3.sql False         26.689338   0.010912   53.378677
12a-unguarded-4.sql False         27.926957   2.311924   55.853914
12a-unguarded-5.sql False         30.508652   1.575468   61.017304
12a-unguarded-6.sql False         29.021063   1.902136   58.042126
13a-unguarded-2.sql False         44.829874   2.872393   89.659749
13a-unguarded-3.sql False         45.158842   0.186317   90.317684
13a-unguarded-4.sql False         42.394087   2.845992   84.788174
13a-unguarded-5.sql False         48.166545   3.260219   96.333090
13a-unguarded-6.sql False         46.222642   2.930512   92.445284
14a-unguarded-2.sql False          9.439458   0.373292   18.878916
14a-unguarded-3.sql False         10.732128   0.326914   21.464256
14a-unguarded-4.sql False         10.390060   1.174138   20.780121
14a-unguarded-5.sql False         11.331828   0.232655   22.663657
14a-unguarded-6.sql False         10.535939   0.433524   21.071877
15a-unguarded-2.sql False          5.171757   0.025270   10.343513
15a-unguarded-3.sql False          6.235897   0.975833   12.471793
15a-unguarded-4.sql False          5.226307   0.366839   10.452615
15a-unguarded-5.sql False          5.840873   0.912258   11.681746
15a-unguarded-6.sql False          5.553413   0.339115   11.106827
16a-unguarded-2.sql False        255.950768  13.417706  511.901537
16a-unguarded-3.sql False        320.861327   1.534145  641.722654
16a-unguarded-4.sql False        301.769408   6.974981  603.538816
16a-unguarded-5.sql False        386.191252  20.208403  772.382505
16a-unguarded-6.sql False        429.527285  20.390636  859.054570
17a-unguarded-2.sql False        189.674494  16.076908  379.348988
17a-unguarded-3.sql False        254.828954   5.280192  509.657908
17a-unguarded-4.sql False        191.554036   0.096650  383.108072
17a-unguarded-5.sql False        252.017841  28.580928  504.035681
18a-unguarded-2.sql False         56.900200   5.915395  113.800400
18a-unguarded-3.sql False         63.611733   2.781871  127.223466
18a-unguarded-4.sql False         53.358576   1.280842  106.717152
18a-unguarded-5.sql False         60.551930   4.732890  121.103860
19a-unguarded-2.sql False         25.542434   4.396463   51.084868
19a-unguarded-3.sql False         28.339473   1.096465   56.678946
19a-unguarded-4.sql False         22.795099   0.690603   45.590198
19a-unguarded-5.sql False         27.528859   2.140198   55.057718
19a-unguarded-6.sql False         26.813629   0.503551   53.627257
1a-unguarded-2.sql  False         17.098070   0.423779   34.196140
1a-unguarded-3.sql  False         20.692089   0.654505   41.384178
1a-unguarded-4.sql  False         16.691897   0.246232   33.383795
20a-unguarded-2.sql False         21.374506   1.292055   42.749011
20a-unguarded-3.sql False         24.254418   2.046566   48.508836
20a-unguarded-4.sql False         22.816610   1.171259   45.633220
20a-unguarded-5.sql False         26.675806   1.852626   53.351612
20a-unguarded-6.sql False         25.879274   1.547501   51.758547
21a-unguarded-5.sql False          8.549363   2.254495   17.098726
21a-unguarded-6.sql False          7.602932   0.551892   15.205864
22a-unguarded-2.

In [58]:
df_compare = df_ref.merge(df_opt, on = 'query', suffixes=('_opt', '_ref'))
df_compare['speedup'] = df_compare[('runtime_opt', 'mean')] / df_compare[('runtime_ref', 'mean')]
df_compare

runtime_opt                        runtime_ref             \
                           mean        std         sum        mean        std   
query                                                                           
10a-unguarded-2.sql    6.173972   0.044003   12.347944    8.620857   2.003390   
10a-unguarded-3.sql    6.548627   0.128008   13.097255    9.919663   0.043037   
10a-unguarded-4.sql    5.638836   0.459086   11.277672    7.445304   2.488442   
10a-unguarded-5.sql    7.104491   0.239550   14.208982    8.017644   2.776277   
11a-unguarded-2.sql    5.831421   0.127936   11.662842    5.785589   0.065313   
11a-unguarded-3.sql    6.092713   0.345270   12.185427    5.840309   0.642704   
11a-unguarded-4.sql    6.308288   1.045292   12.616575    8.668871   0.203665   
11a-unguarded-5.sql    7.242555   0.840189   14.485111    6.094734   0.180450   
11a-unguarded-6.sql    7.129232   0.082742   14.258464    7.077333   0.527993   
12a-unguarded-2.sql   27.494178   1.776018   54.988356   22.604516   0.141099   
12a-unguarded-3.sql   26.689338   0.010912   53.378677   26.513610   1.129923   
12a-unguarded-4.sql   27.926957   2.311924   55.853914   25.842623   0.269888   
12a-unguarded-5.sql   30.508652   1.575468   61.017304   27.251851   0.785090   
12a-unguarded-6.sql   29.021063   1.902136   58.042126   28.992384   0.795900   
13a-unguarded-2.sql   44.829874   2.872393   89.659749   38.145303   0.755204   
13a-unguarded-3.sql   45.158842   0.186317   90.317684   41.462779   1.728428   
13a-unguarded-4.sql   42.394087   2.845992   84.788174   29.436788   0.579433   
13a-unguarded-5.sql   48.166545   3.260219   96.333090   34.874210   1.462016   
13a-unguarded-6.sql   46.222642   2.930512   92.445284   34.635201   1.803636   
14a-unguarded-2.sql    9.439458   0.373292   18.878916    8.577593   0.288746   
14a-unguarded-3.sql   10.732128   0.326914   21.464256    8.859225   0.089177   
14a-unguarded-4.sql   10.390060   1.174138   20.780121    8.269344   0.157789   
14a-unguarded-5.sql   11.331828   0.232655   22.663657   11.392181   0.142381   
14a-unguarded-6.sql   10.535939   0.433524   21.071877    9.445140   0.334737   
15a-unguarded-2.sql    5.171757   0.025270   10.343513    8.534110   0.536918   
15a-unguarded-3.sql    6.235897   0.975833   12.471793    7.584229   2.777802   
15a-unguarded-4.sql    5.226307   0.366839   10.452615    7.080052   2.143329   
15a-unguarded-5.sql    5.840873   0.912258   11.681746   10.718936   1.182072   
15a-unguarded-6.sql    5.553413   0.339115   11.106827    6.945267   2.073282   
16a-unguarded-2.sql  255.950768  13.417706  511.901537   49.605159  19.634838   
16a-unguarded-3.sql  320.861327   1.534145  641.722654   63.682727   3.654268   
16a-unguarded-4.sql  301.769408   6.974981  603.538816   70.292636   5.636997   
16a-unguarded-5.sql  386.191252  20.208403  772.382505   80.458133   5.783902   
16a-unguarded-6.sql  429.527285  20.390636  859.054570   78.322031   2.420909   
17a-unguarded-2.sql  189.674494  16.076908  379.348988   38.480951   0.241154   
17a-unguarded-3.sql  254.828954   5.280192  509.657908   52.357776  16.367695   
17a-unguarded-4.sql  191.554036   0.096650  383.108072   46.327928   9.115740   
17a-unguarded-5.sql  252.017841  28.580928  504.035681   49.104682   0.842367   
18a-unguarded-2.sql   56.900200   5.915395  113.800400   42.062753   2.842985   
18a-unguarded-3.sql   63.611733   2.781871  127.223466   35.143701   0.290398   
18a-unguarded-4.sql   53.358576   1.280842  106.717152   40.570921   2.292467   
18a-unguarded-5.sql   60.551930   4.732890  121.103860   47.170008  14.879875   
19a-unguarded-2.sql   25.542434   4.396463   51.084868   21.142730   1.597233   
19a-unguarded-3.sql   28.339473   1.096465   56.678946   22.052984   1.971788   
19a-unguarded-4.sql   22.795099   0.690603   45.590198   24.768657   1.365658   
19a-unguarded-5.sql   27.528859   2.140198   55.057718   24.037458   0.726359   
19a-unguarded-6.sql   26.813629   0.503551   53.627

In [14]:
df_opt = gen_stats('benchmark-results-lsqb')
df_opt

FileNotFoundError: [Errno 2] No such file or directory: 'benchmark-results-lsqb/results-opt.csv'

In [123]:
df_opt = gen_stats('benchmark-results-lsqb', results='ref', ignore_first=False)
df_opt.round(decimals=2)

runtime                 
                       mean     std      sum
query  physical_cj                          
q1.sql False        3095.81  232.44  6191.62

In [78]:
df_ref = gen_stats_ref('benchmark-results-lsqb')
df_ref

runtime                        
            mean        std          sum
query                                   
q4.sql  602.2586  37.050436  3011.293002

In [86]:
df_opt = gen_stats('benchmark-results-tpch-sf100')
df_opt.round(decimals=2)

runtime              
                                 mean   std     sum
query             physical_cj                      
11-hint.sql       False        164.58  5.90  822.88
                  True         166.96  4.60  834.82
11.sql            False        162.79  3.84  813.94
                  True         167.86  5.79  839.30
2.sql             True          81.54  2.66  407.68
median-1-hint.sql False         50.74  1.90  253.71
                  True          49.67  1.15  248.37
median-1.sql      False         50.04  1.34  250.18
                  True          51.56  0.89  257.78

In [116]:
df_opt = gen_stats('benchmark-results-tpch')
df_opt.round(decimals=2)

runtime                
                                 mean    std      sum
query             physical_cj                        
11-hint.sql       False        346.31   9.74  1731.57
                  True         344.01  10.95  1720.04
11.sql            False        334.92   8.20  1674.62
                  True         329.42   5.92  1647.09
2.sql             False        164.21   4.74   821.07
                  True         160.64   3.74   803.22
median-1-hint.sql False        106.18   1.12   530.92
                  True         102.04   3.88   510.18
median-1.sql      False        107.51   8.90   537.53
                  True         105.11   3.86   525.53

In [111]:
df_opt = gen_stats('benchmark-results-tpch', results='ref')
df_opt.round(decimals=2)

runtime                
                                 mean    std      sum
query             physical_cj                        
11-hint.sql       False        351.76  14.56  1758.79
11.sql            False        351.14  11.91  1755.69
2.sql             False        179.43   6.50   897.17
median-1-hint.sql False        165.93   2.50   829.66
median-1.sql      False        168.44   4.41   842.21

In [51]:
df_opt = gen_stats('benchmark-results-tpch-part1')
df_opt

runtime                      
                         mean       std         sum
query                                              
11-hint.sql        189.647959  3.507685  948.239795
11.sql                    NaN       NaN    0.000000
2.sql               79.777111  3.000262  398.885554
median-1-hint.sql   52.460698  1.423260  262.303489
median-1.sql        91.899140  2.280504  459.495702

In [55]:
df_opt = gen_stats('benchmark-results-snap')
df_opt.round(decimals=2)

runtime              
                      mean   std     sum
query                                   
dblp-path02.sql       1.31  0.18    6.55
dblp-path03.sql       1.59  0.12    7.97
dblp-path04.sql       1.76  0.16    8.82
dblp-path05.sql       2.03  0.25   10.16
dblp-path06.sql       2.18  0.14   10.92
dblp-path07.sql       2.38  0.26   11.92
dblp-path08.sql       2.53  0.30   12.64
dblp-tree01.sql       1.47  0.28    7.37
dblp-tree02.sql       1.69  0.16    8.46
dblp-tree03.sql       1.99  0.16    9.93
google-path02.sql     5.10  0.52   25.48
google-path03.sql     6.08  0.65   30.39
google-path04.sql     6.89  0.30   34.46
google-path05.sql     7.53  0.48   37.67
google-path06.sql     8.80  0.25   43.98
google-path07.sql     9.76  1.21   48.80
google-path08.sql    10.05  1.49   50.26
google-tree01.sql     6.53  1.11   32.65
google-tree02.sql     7.29  0.73   36.44
google-tree03.sql     8.16  0.66   40.82
patents-path02.sql   12.03  0.70   60.14
patents-path03.sql   13.01  0.58   65.06
patents-path04.sql   14.36  0.59   71.82
patents-path05.sql   15.32  1.09   76.58
patents-path06.sql   15.97  1.15   79.85
patents-path07.sql   17.66  0.56   88.32
patents-path08.sql   18.47  1.20   92.35
patents-tree01.sql   14.01  0.75   70.06
patents-tree02.sql   14.38  1.19   71.89
patents-tree03.sql   16.07  1.32   80.35
wiki-path02.sql      21.38  0.68  106.91
wiki-path03.sql      23.71  0.54  118.56
wiki-path04.sql      25.94  1.12  129.70
wiki-path05.sql      27.46  0.64  137.32
wiki-path06.sql      30.16  1.00  150.80
wiki-path07.sql      33.32  1.62  166.61
wiki-path08.sql      34.49  0.68  172.43
wiki-tree01.sql      25.44  0.43  127.20
wiki-tree02.sql      27.64  0.57  138.19
wiki-tree03.sql      30.70  1.01  153.49

In [148]:
pd.set_option('display.max_rows', 500)
df = gen_stats('benchmark-results-hetio', ignore_first=False)
df.round(decimals=2)

runtime             
                                   mean   std    sum
query               physical_cj                     
2-01-CbGaD.sql      False          0.51   NaN   0.51
                    True           0.39  0.02   1.17
2-02-CdGuD.sql      False          0.80   NaN   0.80
                    True           0.66  0.08   1.98
2-03-CrCtD.sql      False          0.32   NaN   0.32
                    True           0.29  0.03   0.88
2-04-CtDrD.sql      False          0.33   NaN   0.33
                    True           0.23  0.02   0.70
2-05-CuGdD.sql      False          0.85   NaN   0.85
                    True           0.62  0.06   1.85
3-01-CbGbCtD.sql    False          0.54   NaN   0.54
                    True           0.47  0.05   1.41
3-02-CbGeAlD.sql    False          2.34   NaN   2.34
                    True           1.51  0.10   3.02
3-03-CbGiGaD.sql    False          1.50   NaN   1.50
                    True           0.93   NaN   0.93
3-04-CcSEcCtD.sql   False          1.16   NaN   1.16
                    True           0.90   NaN   0.90
3-05-CdGdCtD.sql    False          1.06   NaN   1.06
                    True           0.70   NaN   0.70
3-06-CdGuCtD.sql    False          0.79   NaN   0.79
                    True           0.65   NaN   0.65
3-07-CiPCiCtD.sql   False          2.92   NaN   2.92
                    True           4.84   NaN   4.84
3-08-CpDpCtD.sql    False          0.33   NaN   0.33
                    True           0.35   NaN   0.35
3-09-CrCrCtD.sql    False          0.39   NaN   0.39
                    True           0.38   NaN   0.38
3-10-CrCbGaD.sql    False          0.41   NaN   0.41
                    True           0.51   NaN   0.51
3-11-CtDdGdD.sql    False          1.02   NaN   1.02
                    True           0.60   NaN   0.60
3-12-CtDdGuD.sql    False          0.78   NaN   0.78
                    True           0.56   NaN   0.56
3-13-CtDlAlD.sql    False          0.33   NaN   0.33
                    True           0.31   NaN   0.31
3-14-CtDpSpD.sql    False          0.50   NaN   0.50
                    True           0.30   NaN   0.30
3-15-CtDuGdD.sql    False          0.94   NaN   0.94
                    True           0.74   NaN   0.74
3-16-CtDuGuD.sql    False          0.89   NaN   0.89
                    True           0.68   NaN   0.68
3-17-CuGdCtD.sql    False          0.80   NaN   0.80
                    True           0.71   NaN   0.71
3-18-CuGuCtD.sql    False          0.76   NaN   0.76
                    True           0.64   NaN   0.64
4-01-CbGiGiGaD.sql  False          1.28   NaN   1.28
                    True           1.35   NaN   1.35
4-02-CbGpBPpGaD.sql False          3.95   NaN   3.95
                    True           4.29   NaN   4.29
4-03-CbGpPWpGaD.sql False          3.86   NaN   3.86
                    True           2.44   NaN   2.44
CbGiGiGaD.sql       False          1.40  0.17   7.00
                    True           1.35  0.22   6.77
CbGpBPpGaD.sql      False          3.83  0.31  30.64
                    True           3.43  0.54  27.46
CbGpPWpGaD.sql      False          3.02  0.35  24.13
                    True           2.71  0.30  21.65
CtDpSpD.sql         False          0.42  0.07   1.27
                    True           0.35  0.05   1.06
CtDuGdD.sql         False          0.85  0.16   2.56
                    True           0.62  0.03   1.86
CtDuGuD.sql         False          0.86  0.09   2.58
                    True           0.61  0.06   1.83
CuGdCtD.sql         False          0.77  0.22   2.32
                    True           0.64  0.07   1.91
CuGuCtD.sql         False          1.01  0.06   3.04
                    True           0.66  0.10   1.98
q1.sql              False          3.36  0.49  20.16
                    True           2.78  0.43  16.70

In [149]:
df = gen_stats('benchmark-results-hetio', results='ref', ignore_first=False)
df.round(decimals=2)

runtime               
                                   mean    std     sum
query               physical_cj                       
2-01-CbGaD.sql      False          0.36    NaN    0.36
2-02-CdGuD.sql      False          0.66    NaN    0.66
2-03-CrCtD.sql      False          0.21    NaN    0.21
2-04-CtDrD.sql      False          0.18    NaN    0.18
2-05-CuGdD.sql      False          0.57    NaN    0.57
3-01-CbGbCtD.sql    False          0.57    NaN    0.57
3-02-CbGeAlD.sql    False          2.06    NaN    2.06
3-03-CbGiGaD.sql    False          0.53    NaN    0.53
3-04-CcSEcCtD.sql   False         12.21    NaN   12.21
3-05-CdGdCtD.sql    False          1.14    NaN    1.14
3-06-CdGuCtD.sql    False          0.56    NaN    0.56
3-07-CiPCiCtD.sql   False          0.85    NaN    0.85
3-08-CpDpCtD.sql    False          0.25    NaN    0.25
3-09-CrCrCtD.sql    False          0.31    NaN    0.31
3-10-CrCbGaD.sql    False          0.32    NaN    0.32
3-11-CtDdGdD.sql    False          0.79    NaN    0.79
3-12-CtDdGuD.sql    False          0.60    NaN    0.60
3-13-CtDlAlD.sql    False          0.22    NaN    0.22
3-14-CtDpSpD.sql    False          0.23    NaN    0.23
3-15-CtDuGdD.sql    False          0.59    NaN    0.59
3-16-CtDuGuD.sql    False          0.54    NaN    0.54
3-17-CuGdCtD.sql    False          0.49    NaN    0.49
3-18-CuGuCtD.sql    False          0.57    NaN    0.57
4-01-CbGiGiGaD.sql  False         19.01    NaN   19.01
4-02-CbGpBPpGaD.sql False         52.92    NaN   52.92
4-03-CbGpPWpGaD.sql False         11.18    NaN   11.18
CbGiGiGaD.sql       False         16.43   1.64   82.17
CbGpBPpGaD.sql      False         58.57  12.98  468.56
CbGpPWpGaD.sql      False         12.75   1.29  102.01
CtDpSpD.sql         False          0.23   0.01    0.70
CtDuGdD.sql         False          0.65   0.07    1.95
CtDuGuD.sql         False          0.68   0.10    2.04
CuGdCtD.sql         False          0.52   0.03    1.55
CuGuCtD.sql         False          0.85   0.15    2.56
q1.sql              False         12.10   1.64   72.58

In [ ]:
stats_ref_df = pd.read_csv('benchmark-results-stats/results-ref.csv')
df_ref = stats_ref_df[stats_opt_df['run'] != 1]
df_ref = df_ref.groupby(['query']).agg({'runtime': ['mean', 'std', 'sum']})
df_ref.to_csv('benchmark-results-stats/results-ref-mean.csv')

In [ ]:
#df[['query', 'runtime']].plot()
df = df_opt
plt.plot(df[df.index.str.contains('hint.sql')][[('runtime', 'mean')]])
plt.plot(df[~df.index.str.contains('hint.sql')][[('runtime', 'mean')]])
#df.index#['query']

In [ ]:
df_opt[df_opt.index.str.contains('hint.sql')][[('runtime', 'sum')]].sum()

In [ ]:
df_opt[~df_opt.index.str.contains('hint.sql')][[('runtime', 'sum')]].sum()

In [ ]:
df_ref[[('runtime', 'sum')]].sum()

In [ ]:
df = pd.read_csv('benchmark-results-tpch/results-ref.csv')

In [ ]:
print(df)

In [7]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.read_csv('results-neo4j.csv')
df['runtime'] = df['runtime'] / 1000
df_agg = df.groupby('query').agg({'runtime': ['mean', 'std', 'sum']})
print(df_agg)

               runtime                 
                  mean     std      sum
query                                  
dblp-path03     97.792   0.729  391.168
dblp-path04   1725.101  81.605 6900.406
google-path03 1686.677 114.285 6746.707


In [5]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.read_csv('results-graphdb.csv')
df['runtime'] = df['runtime'] / 1000
df_agg = df.groupby('query').agg({'runtime': ['mean', 'std', 'sum']})
print(df_agg)

            runtime               
               mean   std      sum
query                             
dblp-path03 322.750 0.957 1291.000
